<a href="https://colab.research.google.com/github/ahteshamsalamatansari/colabcodes/blob/main/logoex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!google-chrome --version

In [ ]:
!wget -q -O chromedriver.zip https://storage.googleapis.com/chrome-for-testing-public/140.0.7339.207/linux64/chromedriver-linux64.zip
!unzip -o chromedriver.zip
!mv -f chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver
!rm -rf chromedriver-linux64 chromedriver.zip
!pip install selenium --quiet

In [ ]:
!chromedriver --version
!google-chrome --version

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920,1200')
chrome_options.binary_location = '/usr/bin/google-chrome'

driver = webdriver.Chrome(service=Service('/usr/bin/chromedriver'), options=chrome_options)
url = 'https://vivatechnology.com/partners-wall#partners_exhibitor_partners'
driver.get(url)
time.sleep(10)  # Wait for JS-heavy content to fully load (increase if needed)

# Scroll to "Exhibitor Partners" (optional, helps lazy-load)
try:
    section = driver.find_element(By.XPATH, "//*[contains(text(), 'Exhibitor Partners')]")
    driver.execute_script("arguments[0].scrollIntoView(true);", section)
    time.sleep(3)
except Exception:
    pass

# ----------- Scrape all <li> entries containing <img alt=...> under exhibitor partners -----------
logo_data = []
# Multiple XPath strategies for robustness
selectors = [
    "//h4[contains(text(), 'Exhibitor Partners')]/following-sibling::*//li[.//img[@alt]]",
    "//h3[contains(text(), 'Exhibitor Partners')]/following-sibling::*//li[.//img[@alt]]",
    "//li[.//img[@alt and string-length(@alt) > 2]]"
]
found = False
for sel in selectors:
    lis = driver.find_elements(By.XPATH, sel)
    if len(lis) >= 5:  # likely correct parent if section is loaded
        for li in lis:
            imgs = li.find_elements(By.TAG_NAME, 'img')
            for img in imgs:
                alt = img.get_attribute('alt')
                src = img.get_attribute('src')
                if alt and len(alt.strip()) > 2:
                    logo_data.append({'alt_text': alt.strip(), 'img_src': src})
        found = True
        break

if not found:
    imgs = driver.find_elements(By.XPATH, "//img[@alt and string-length(@alt) > 2]")
    for img in imgs:
        alt = img.get_attribute('alt')
        src = img.get_attribute('src')
        if alt:
            logo_data.append({'alt_text': alt.strip(), 'img_src': src})

driver.quit()

# 4. Save and show results
df = pd.DataFrame(logo_data).drop_duplicates()
csvfile = "vivatech_exhibitor_partners_logos.csv"
df.to_csv(csvfile, index=False, encoding='utf-8')
print(f"✅ Found {len(df)} logo alt texts. Download below:")

from IPython.display import display, HTML
display(df.head(10))
display(HTML(f'<a href=\"{csvfile}\" download>📥 Download CSV</a>'))

if len(df) > 0:
    print("\nAlt Texts Extracted:")
    for i, row in df.head(15).iterrows():
        print(f"{i+1:2d}. {row['alt_text']}")
    if len(df) > 15:
        print(f"...and {len(df)-15} more.")
else:
    print("❌ No alt texts found. Try increasing time.sleep, or check if section loads properly.")